In [1]:
import numpy as onp  # Original NumPy for data generation
import autograd.numpy as np  # Autograd NumPy for manifold optimization
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score, precision_score
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
from scipy import linalg
from numpy.linalg import inv, det
#!pip install pymanopt
from pymanopt.manifolds import Grassmann, Euclidean, Product
from pymanopt import Problem
from pymanopt.optimizers import SteepestDescent
import pymanopt.function
from scipy.linalg import svd, qr, pinv, eigh
import time
from scipy.linalg import lu_factor, lu_solve
from scipy.sparse.linalg import cg
from scipy.sparse.linalg import svds, aslinearoperator, LinearOperator
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
import warnings
import numpy as np

def delta_kernel(Y):
    n = len(Y)
    K = onp.zeros((n, n))
    for i in range(n):
        for j in range(n):
            K[i, j] = 1 if Y[i] == Y[j] else 0
    return K


def barshan_pca(X, Y, K, q, epsilon= 1e-6):
    # Cholesky decomposition of K
    n = K.shape[0]
    K_reg = K + epsilon * np.eye(n)
    Delta = linalg.cholesky(K_reg, lower=True)  # K = Delta @ Delta.T
    psi = X.T @ Delta  # Shape: (p x n)
    # SVD of psi
    U, s, Vt = svd(psi, full_matrices=False)
    Sigma = onp.diag(s)
    # Extract top q components
    U_d = U[:, :q]
    Sigma_d = Sigma[:q, :q]
    V_d = Vt.T[:, :q]  # Vt is (n x p), so V_d is (p x q)
    # Compute U_hat
    U_hat = psi @ V_d @ linalg.inv(Sigma_d)  # Shape: (p x q)
    eigenvalues = s[:q]**2  # Singular values squared
    return {'W': U_hat, 'singular_values': eigenvalues}



# Supervised PCA (Updated)
def supervised_pca(X, Y, q, p, m, lambda_):
    XtX = X.T @ X
    C = X.T @ delta_kernel(Y) @ X + lambda_ * XtX
    # Subsample m features
    indices = onp.random.choice(p, size=m, replace=False)
    C_nm = C[:, indices]  # Shape: (p x m)
    C_mm = C[indices, :][:, indices]  # Shape: (m x m)
    # Eigen decomposition of C_mm
    eigenvalues, U_m = linalg.eigh(C_mm)  # Symmetric matrix
    sorted_indices = onp.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    U_m = U_m[:, sorted_indices]
    # Compute Lambda_m and its inverse square root
    Lambda_m = onp.diag(eigenvalues)
    Lambda_m_inv_sqrt = onp.diag(1.0 / onp.sqrt(onp.maximum(eigenvalues, 1e-6)))
    # Compute U_n
    U_n = C_nm @ U_m @ Lambda_m_inv_sqrt  # Shape: (p x m)
    # Compute approximate projection
    C_mm_inv = pinv(C_mm)  # Generalized inverse
    C_approx = C_nm @ C_mm_inv @ C_nm.T
    # Extract top q components and orthogonalize
    W = U_n[:, :q]
    W, _ = qr(W, mode='economic')  # Orthogonalize W
    return {'W': W, 'eigenvalues': eigenvalues[:q]}




def proximal_l1(b, lambda_, q):
    Z = np.sign(b) * np.maximum(np.abs(b) - lambda_, 0)
    act_set = (Z != 0).astype(int)
    inact_set = 1 - act_set
    return Z, act_set, inact_set

#The following function generates a duplication matrix Dn and its pseudo-inverse pDn to be used later in the RSSN method.
def duplication_matrix(q):
    n = q * (q + 1) // 2
    Dn = np.zeros((q * q, n))
    idx = 0
    for i in range(q):
        for j in range(i, q):
            Dn[i * q + j, idx] = 1
            if i != j:
                Dn[j * q + i, idx] = 1
            idx += 1
    pDn = np.linalg.pinv(Dn)
    return Dn, pDn

#Applies the Jacobian of the residual equation
def linop(Blkd, x, q, t, reg):
    V = np.zeros_like(x)
    for i in range(q):
        V[:, i] = Blkd[i] @ x[:, i]
    return 2 * t * (V + V.T) + reg * x

#The following function performs the optimization using the semi-smooth newton algorithm
def semi_newton_matrix(n, q, W, t, C, mut, inner_tol, inner_max_iter, Lam0, Dn, pDn):
    WtW = np.eye(q)
    Wt = W.T
    stop_flag = 0
    Lam = Lam0.copy()
    W_Lam_prod = C + 2 * t * (W @ Lam)
    Z, Act_set, Inact_set = proximal_l1(W_Lam_prod, mut, q)
    ZW = Z.T @ W
    R_Lam = ZW + ZW.T - 2 * np.eye(q)
    RE = pDn @ R_Lam.flatten()
    r_l = np.linalg.norm(R_Lam, 'fro')

    lambda_ = 0.2
    j = 0

    while r_l**2 > inner_tol:
        reg = lambda_ * max(min(r_l, 0.1), 1e-11)
        nnzZ = np.count_nonzero(Z)

        if q < 15:
            if nnzZ > q * (q + 1) // 2:
                g = np.zeros((q * q, q * q))
                for i in range(q):
                    g[i*q:(i+1)*q, i*q:(i+1)*q] = Wt @ (Act_set[:, i, None] * W)
                G = 4 * t * (pDn @ (g @ Dn))
                lu, piv = lu_factor(G + reg * np.eye(q * (q + 1) // 2))
                new_d = -lu_solve((lu, piv), RE)
            else:
                Wstack = np.zeros((nnzZ, q * q))
                dim = 0
                for i in range(q):
                    row = np.where(Act_set[:, i])[0]
                    Wstack[dim:dim+len(row), i*q:(i+1)*q] = W[row, :]
                    dim += len(row)
                V = Wstack @ Dn
                U = 4 * t * (pDn @ Wstack.T)
                lu, piv = lu_factor(np.eye(nnzZ) + (1/reg) * (V @ U))
                new_d = -(1/reg * RE - (1/reg**2) * U @ lu_solve((lu, piv), V @ RE))
            new_d = Dn @ new_d
            new_d = new_d.reshape(q, q)
        else:
            Blkd = [None] * q
            for i in range(q):
                ind = Act_set[:, i].astype(bool)
                if np.sum(ind) < n / 2:
                    W_ind = W[ind, :]
                    Blkd[i] = W_ind.T @ W_ind
                else:
                    ind = Inact_set[:, i].astype(bool)
                    W_ind = Wt[:, ind]
                    Blkd[i] = WtW - W_ind @ W_ind.T
            new_d, _ = cg(lambda x: linop(Blkd, x, q, t, reg), -R_Lam, tol=min(1e-4, 1e-3 * r_l))
            new_d = new_d.reshape(q, q)

        t_new = 1.0
        W_d_prod = 2 * t * (W @ new_d)
        W_Lam_new_prod = W_Lam_prod + t_new * W_d_prod
        Z, Act_set, Inact_set = proximal_l1(W_Lam_new_prod, mut, q)
        ZW = Z.T @ W
        R_Lam_new = ZW + ZW.T - 2 * np.eye(q)
        r_l_new = np.linalg.norm(R_Lam_new, 'fro')

        while r_l_new**2 >= r_l**2 * (1 - 0.001 * t_new) and t_new > 1e-3:
            t_new *= 0.5
            W_Lam_new_prod = W_Lam_prod + t_new * W_d_prod
            Z, Act_set, Inact_set = proximal_l1(W_Lam_new_prod, mut, q)
            ZW = Z.T @ W
            R_Lam_new = ZW + ZW.T - 2 * np.eye(q)
            r_l_new = np.linalg.norm(R_Lam_new, 'fro')

        Lam += t_new * new_d
        r_l = r_l_new
        R_Lam = R_Lam_new
        RE = pDn @ R_Lam.flatten()
        W_Lam_prod = W_Lam_new_prod

        if j > inner_max_iter:
            stop_flag = 1
            break
        j += 1

    return Z, j, Lam, r_l, stop_flag

#This is the ManPG algorithm adapted specifically for sparse CSPCA. The projection matrix W is initialised based on the standard CSPCA method.
def manpg_orth_sparse(C, q, n, eta, maxiter=1000, tol=1e-6, inner_iter=100, phi_init=None):
    start_time = time.time()

    #eta corresponds to the sparsity parameter, phi is defined as the solution to the standard CSPCA problem.
    eta = eta.reshape(-1, 1)
    if phi_init is None:
      eigenvalues, eigenvectors = eigh(C)
      phi_init = eigenvectors[:, -q:]

    Dn, pDn = duplication_matrix(q)
    L = 2 * abs(max(eigenvalues))
    t = 2 / L
    t_min = 1e-4
    #tol = tol * n * q increase the tolerance based on the original paper!

    # Initialization
    W = phi_init.copy()
    AW = C @ W
    F = [-np.sum(W * AW) + np.sum(eta * np.abs(W))]
    num_inner = np.zeros(maxiter)
    num_linesearch = 0
    num_inexact = 0
    alpha = 1.0

    for iter_ in range(1, maxiter):
        ngx = 2 * AW
        neg_pgx = ngx

        if alpha < t_min or num_inexact > 10:
            inner_tol = max(5e-16, min(1e-14, 1e-5 * tol * t**2))
        else:
            inner_tol = max(1e-13, min(1e-11, 1e-3 * tol * t**2))

        Lam_init = np.zeros((q, q)) if iter_ == 1 else Lam
        PY, num_inner[iter_], Lam, _, in_flag = semi_newton_matrix(
            n, q, W, t, W + t * neg_pgx, eta * t, inner_tol, inner_iter, Lam_init, Dn, pDn
        )
        if in_flag:
            num_inexact += 1

        alpha = 1.0
        D = PY - W
        def orthonormalize_stable(PY, epsilon=1e-8):
            M = PY.T @ PY + epsilon * np.eye(PY.shape[1])
            U, Sigma, Vt = svd(M, full_matrices=False)
            Sigma_inv_sqrt = np.diag([1 / np.sqrt(s) if s > epsilon else 0.0 for s in Sigma])
            return PY @ (U @ Sigma_inv_sqrt @ Vt)

        Z = orthonormalize_stable(PY)
        #U, Sigma, Vt = svd(PY.T @ PY, full_matrices=False)
        #Z = PY @ (U @ np.diag(np.sqrt(1 / Sigma)) @ Vt)
        AZ = C @ Z
        f_trial = -np.sum(Z * AZ)
        F_trial = f_trial + np.sum(eta * np.abs(Z))
        normDsquared = np.linalg.norm(D, 'fro')**2

        if normDsquared / t**2 < tol:
            break

        while F_trial >= F[-1] - 0.5 / t * alpha * normDsquared:
            alpha *= 0.5
            num_linesearch += 1
            if alpha < t_min:
                num_inexact += 1
                break
            PY = W + alpha * D
            Z = orthonormalize_stable(PY)
            #U, Sigma, Vt = svd(PY.T @ PY, full_matrices=False)
            #Z = PY @ (U @ np.diag(np.sqrt(1 / Sigma)) @ Vt)
            AZ = C @ Z
            f_trial = -np.sum(Z * AZ)
            F_trial = f_trial + np.sum(eta * np.abs(Z))

        W = Z
        AW = AZ
        F.append(F_trial)

    #W[np.abs(W) <= 1e-4] = 0
    W_manpg = W
    time_manpg = time.time() - start_time
    mean_ssn = np.sum(num_inner) / iter_

    sparsity = np.count_nonzero(W_manpg == 0) / (n * q)
    F_manpg = F[-1]
    flag_succ = 1 if iter_ < maxiter - 1 else 0

    print(f"ManPG: Iter {iter_}, Fval {min(F):.5e}, CPU {time_manpg:.2f}, "
          f"Sparsity {sparsity:.2f}, Inner Inexact {num_inexact}, "
          f"Avg Inner {mean_ssn:.2f}, Total Linesearch {num_linesearch}")

    return W_manpg, F_manpg, sparsity, time_manpg, iter_, flag_succ, num_linesearch, mean_ssn

# SPLS and SPCA functions (provided by user)
def ust(b, eta):
    b_ust = np.zeros_like(b)
    if eta < 1:
        valb = np.abs(b) - eta * np.max(np.abs(b))
        mask = valb >= 0
        b_ust[mask] = valb[mask] * np.sign(b)[mask]
    return b_ust


def rootmatrix(x):
    eigvals, eigvecs = np.linalg.eigh(x)
    eigvals = (eigvals + np.abs(eigvals)) / 2
    sqrt_d = np.sqrt(eigvals)
    return eigvecs @ np.diag(sqrt_d) @ eigvecs.T

def convcheck(beta1, beta2):
    a = np.max(np.abs(beta1 + beta2), axis=0)
    b = np.max(np.abs(beta1 - beta2), axis=0)
    d = len(a)
    x = np.minimum(a, b)
    return np.max(x)

def updateRR(xnew, R=None, xold=None, lambda_=0, eps=1e-16):
    xtx = (np.sum(xnew**2) + lambda_) / (1 + lambda_)
    norm_xnew = np.sqrt(xtx)
    if R is None:
        R = np.array([[norm_xnew]])
        return R, 1
    Xtx = np.dot(xnew.T, xold) / (1 + lambda_)
    r = linalg.solve_triangular(R, Xtx, lower=False, trans='T')
    rpp = norm_xnew**2 - np.sum(r**2)
    if rpp <= eps:
        rpp = eps
        rank = R.shape[0]
    else:
        rpp = np.sqrt(rpp)
        rank = R.shape[0] + 1
    R_new = np.zeros((R.shape[0] + 1, R.shape[1] + 1))
    R_new[:-1, :-1] = R
    R_new[:-1, -1] = r
    R_new[-1, -1] = rpp
    return R_new, rank

def solvebeta(x, y, paras, max_steps=None, sparse="penalty", eps=1e-16):
    lambda_ = paras[0]
    penalty_param = paras[1]
    n, m = x.shape
    im = np.arange(m)
    one = np.ones(n)
    if lambda_ > 0:
        maxvars = m
    else:
        maxvars = min(m, n - 1)
        if m == n:
            maxvars = m
    if max_steps is None:
        max_steps = 50 * maxvars
    d1 = np.sqrt(lambda_)
    d2 = 1 / np.sqrt(1 + lambda_)
    Cvec = np.dot(y.T, x) * d2
    ssy = np.sum(y**2)
    residuals = np.concatenate([y, np.zeros(m)])
    penalty = np.array([np.max(np.abs(Cvec))])
    if sparse == "penalty" and penalty[0] * 2 / d2 <= penalty_param:
        return np.zeros(m)
    beta = np.zeros(m)
    first_in = np.zeros(m, dtype=int)
    active = []
    ignores = []
    actions = [None] * max_steps
    Sign = []
    R = None
    k = 0
    while k < max_steps and len(active) < maxvars - len(ignores):
        action = []
        k += 1
        inactive = im if k == 1 else np.setdiff1d(im, active + ignores)
        C = Cvec[inactive]
        Cmax = np.max(np.abs(C))
        new = np.abs(C) == Cmax
        C = C[~new]
        new = inactive[new]
        for inew in new:
            R, rank = updateRR(x[:, inew], R, x[:, active] if active else None, lambda_, eps)
            if rank == len(active):
                R = R[:len(active), :len(active)] if len(active) > 0 else np.array([[]])
                ignores.append(inew)
                action.append(-inew)
            else:
                if first_in[inew] == 0:
                    first_in[inew] = k
                active.append(inew)
                Sign.append(np.sign(Cvec[inew]))
                action.append(inew)
        Gi1 = linalg.solve_triangular(R, linalg.solve_triangular(R, Sign, lower=False, trans='T'), lower=False)
        A = 1 / np.sqrt(np.sum(Gi1 * Sign))
        w = A * Gi1
        u1 = np.dot(x[:, active], w) * d2
        u2 = np.zeros(m)
        u2[active] = d1 * d2 * w
        u = np.concatenate([u1, u2])
        if len(active) == maxvars - len(ignores):
            gamhat = Cmax / A
        else:
            a = (np.dot(u1, x[:, np.setdiff1d(im, active + ignores)]) + d1 * u2[np.setdiff1d(im, active + ignores)]) * d2
            gam = np.concatenate([(Cmax - C) / (A - a), (Cmax + C) / (A + a)])
            gamhat = np.min(gam[gam > eps]) if len(gam) > 0 else Cmax / A
        dropid = None
        b1 = beta[active]
        z1 = -b1 / w
        zmin = np.min(z1[z1 > eps]) if np.any(z1 > eps) else gamhat
        drops = False
        if zmin < gamhat:
            gamhat = zmin
            drops = z1 == zmin
        beta2 = beta.copy()
        beta[active] += gamhat * w
        residuals -= gamhat * u
        Cvec = (np.dot(residuals[:n], x) + d1 * residuals[n:]) * d2
        penalty = np.append(penalty, penalty[-1] - np.abs(gamhat * A))
        if sparse == "penalty" and penalty[-1] * 2 / d2 <= penalty_param:
            s1 = penalty[-1] * 2 / d2
            s2 = penalty[-2] * 2 / d2 if len(penalty) > 1 else s1
            beta = ((s2 - penalty_param) / (s2 - s1)) * beta + ((penalty_param - s1) / (s2 - s1)) * beta2
            beta *= d2
            break
        if isinstance(drops, np.ndarray) and drops.any() or drops is True:  # Handle both array and boolean
            dropid = np.where(drops)[0] if isinstance(drops, np.ndarray) else []
            for id in reversed(dropid):
                R = R[:-1, :-1] if R.shape[0] > 1 else np.array([[]])
            dropid = np.array(active)[drops] if isinstance(drops, np.ndarray) else []
            beta[dropid] = 0
            active = [a for i, a in enumerate(active) if not drops[i]] if isinstance(drops, np.ndarray) else active
            Sign = [s for i, s in enumerate(Sign) if not drops[i]] if isinstance(drops, np.ndarray) else Sign
        if sparse == "varnum" and len(active) >= penalty_param:
            break
        actions[k-1] = action
    return beta

def spca(x, K, para, type_="predictor", sparse="penalty", use_corr=False, lambda_=1e-6, max_iter=200, trace=False, eps_conv=1e-3):
    x = np.asarray(x)
    n, p = x.shape if type_ == "predictor" else (None, x.shape[0])

    if type_ == "predictor":
        if n / p >= 100:
            print("Consider using type='Gram' with the sample covariance/correlation matrix for efficiency.")
        mean_x = np.mean(x, axis=0)
        scale_x = np.std(x, axis=0) if use_corr else np.ones(p)
        if use_corr:
            scale_x[scale_x == 0] = 1.0
        x = (x - mean_x) / scale_x
    elif type_ == "Gram":
        x = rootmatrix(x)

    U, s, Vt = linalg.svd(x, full_matrices=False)
    v = Vt.T
    total_variance = np.sum(s**2)
    alpha = v[:, :K]
    beta = alpha.copy()

    for i in range(K):
        y = np.dot(x, alpha[:, i])
        beta[:, i] = solvebeta(x, y, paras=[lambda_, para[i]], sparse=sparse)

    xtx = np.dot(x.T, x)
    temp = beta.copy()
    norm_temp = np.sqrt(np.sum(temp**2, axis=0))
    norm_temp[norm_temp == 0] = 1
    temp = temp / norm_temp

    k = 0
    diff = 1
    while k < max_iter and diff > eps_conv:
        k += 1
        alpha = np.dot(xtx, beta)
        U, _, Vt = linalg.svd(alpha, full_matrices=False)
        alpha = np.dot(U, Vt)

        for i in range(K):
            y = np.dot(x, alpha[:, i])
            beta[:, i] = solvebeta(x, y, paras=[lambda_, para[i]], sparse=sparse)

        norm_beta = np.sqrt(np.sum(beta**2, axis=0))
        norm_beta[norm_beta == 0] = 1
        beta2 = beta / norm_beta
        diff = convcheck(beta2, temp)
        temp = beta2

        if trace and k % 10 == 0:
            print(f"Iterations: {k}")

    norm_beta = np.sqrt(np.sum(beta**2, axis=0))
    norm_beta[norm_beta == 0] = 1
    beta = beta / norm_beta

    u = np.dot(x, beta)
    R = np.linalg.qr(u)[1]
    pev = np.diag(R**2) / total_variance

    result = {
        "type": type_,
        "K": K,
        "loadings": beta,
        "pev": pev,
        "var_all": total_variance,
        "para": para,
        "lambda": lambda_
    }
    return result



class SDSPCA:
    def __init__(self, n_components=2, alpha=1.0, beta=1.0, max_iter=100, tol=1e-6):
        self.n_components = n_components
        self.alpha = alpha
        self.beta = beta
        self.max_iter = max_iter
        self.tol = tol
        
    def _create_label_matrix(self, y):
        """Create the class indicator matrix B from labels y"""
        classes = np.unique(y)
        n_classes = len(classes)
        n_samples = len(y)
        B = np.zeros((n_classes, n_samples))
        
        for i, cls in enumerate(classes):
            B[i, y == cls] = 1
            
        return B
    
    def fit(self, X, y):
        """
        Fit the model to the data.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Training data (scikit-learn convention).
        y : array-like, shape (n_samples,)
            Target labels.
        """
        X = np.asarray(X)
        y = np.asarray(y)
        
        n_samples, n_features = X.shape
        
        # Transpose X to match paper's (m × n) convention
        X = X.T  # Now shape (n_features, n_samples)
        
        # Create label matrix B (c × n)
        self.B_ = self._create_label_matrix(y)
        n_classes = self.B_.shape[0]
        
        # Initialize variables
        self.Q_ = np.random.randn(n_samples, self.n_components)
        self.Q_ = np.linalg.qr(self.Q_)[0]  # Orthogonalize
        self.V_ = np.eye(n_samples)
        self.A_ = np.random.randn(n_classes, self.n_components)
        
        prev_obj = np.inf
        
        for iter_ in range(self.max_iter):
            # Update Y (m × k)
            self.Y_ = X @ self.Q_
            
            # Update A (c × k)
            self.A_ = self.B_ @ self.Q_
            
            # Update Q (n × k)
            # Compute matrix Z = -XᵀX - αBᵀB + βV
            XTX = X.T @ X  # (n × n)
            BTB = self.B_.T @ self.B_  # (n × n)
            Z = -XTX - self.alpha * BTB + self.beta * self.V_  # (n × n)
            
            # Get eigenvectors corresponding to smallest eigenvalues
            eigvals, eigvecs = eigh(Z, subset_by_index=[0, self.n_components-1])
            self.Q_ = eigvecs
            
            # Update V (n × n)
            row_norms = np.linalg.norm(self.Q_, axis=1)
            self.Q_[row_norms < 1e-5, :] = 0 
            self.V_ = np.diag(1. / (2 * np.maximum(row_norms, 1e-16)))
            
            # Check convergence
            current_obj = (
                np.linalg.norm(X - self.Y_ @ self.Q_.T, 'fro')**2 +
                self.alpha * np.linalg.norm(self.B_ - self.A_ @ self.Q_.T, 'fro')**2 +
                self.beta * np.sum(row_norms)
            )
            
            if np.abs(prev_obj - current_obj) < self.tol:
                break
                
            prev_obj = current_obj
            
        self.n_iter_ = iter_ + 1
        
        return self
    


    def transform(self, X):
        """
        Transform data to the principal components.
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Data to transform.
            
        Returns:
        --------
        Q : array-like, shape (n_samples, n_components)
            Transformed data.
        """
#        # Project using the learned Y (X is n_samples × m, Y is m × k)
#        X = np.asarray(X)
#        return X @ self.Q_
        return X @ self.Y_
    
    def fit_transform(self, X, y):
        """Fit the model and transform the data."""
        self.fit(X, y)
        return self.Q_

In [2]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


def soft_threshold(a, tau):
    return np.sign(a) * np.maximum(np.abs(a) - tau, 0)



def find_tau(a, c, max_iter=5000, tol=1e-4):
    # Check if tau = 0 satisfies the L1 constraint
    v_temp = soft_threshold(a, 0)
    if np.linalg.norm(v_temp) > 0:
        v_temp = v_temp / np.linalg.norm(v_temp)
        if np.sum(np.abs(v_temp)) <= c:
            return 0

    # Binary search for tau
    tau_min, tau_max = 0, np.max(np.abs(a))
    best_tau, best_norm_diff = 0, float('inf')

    for _ in range(max_iter):
        tau = (tau_min + tau_max) / 2
        v_temp = soft_threshold(a, tau)
        if np.linalg.norm(v_temp) > 0:
            v_temp = v_temp / np.linalg.norm(v_temp)
            l1_norm = np.sum(np.abs(v_temp))
            norm_diff = abs(l1_norm - c)

            if norm_diff < best_norm_diff:
                best_tau, best_norm_diff = tau, norm_diff

            if norm_diff < tol:
                return tau

            if l1_norm > c:
                tau_min = tau
            else:
                tau_max = tau
        else:
            tau_min = tau  # If v_temp is zero, increase tau

    print(f"Warning: Tau search did not converge within tolerance (best norm diff: {best_norm_diff:.6f})")
    return best_tau


def compute_kernel(Y, kernel_type='linear', sigma=1.0, epsilon=1e-6):
    n = Y.shape[0]
    Y = Y.reshape(-1, 1)  # Ensure Y is a column vector

    if kernel_type == 'linear':
        L = Y @ Y.T  # Linear kernel (for regression)
    elif kernel_type == 'rbf':
        pairwise_dists = np.sum(Y**2, axis=1).reshape(-1, 1) + np.sum(Y**2, axis=1) - 2 * (Y @ Y.T)
        L = np.exp(-pairwise_dists / (2 * sigma**2))  # RBF kernel
    elif kernel_type == 'delta':
        # Delta kernel: 1 if Y_i == Y_j, else 0 (for classification)
        L = (Y == Y.T).astype(float)
    else:
        raise ValueError(f"Unsupported kernel type: {kernel_type}")

    # Add ridge term for numerical stability
    L += epsilon * np.eye(n)
    return L





def sspca(X, Y, K, c, X_test=None, Y_test=None, kernel_type='linear',
          sigma=1.0, max_iter=1000, tol=1e-6, verbose=False, standardize=True):
    """
    Sparse Supervised Principal Component Analysis (SSPCA) with prediction and RMSE.

    Parameters:
    X : ndarray
        Input training data matrix (n x p)
    Y : ndarray
        Target variable (n x 1 or n for classification)
    K : int
        Number of components
    c : float
        L1 regularization parameter (1 <= c <= sqrt(p))
    X_test : ndarray, optional
        Test data matrix (m x p)
    Y_test : ndarray, optional
        Test target variable (m x 1 or m)
    kernel_type : str
        Type of kernel for L ('linear', 'rbf', 'delta')
    sigma : float
        RBF kernel bandwidth
    max_iter : int
        Maximum iterations for convergence
    tol : float
        Convergence tolerance
    verbose : bool
        If True, print iteration details
    standardize : bool
        If True, standardize X and Y

    Returns:
    dict
        Contains:
        - Z: Dimension reduced training data (n x K)
        - V: Sparse eigenvectors (p x K)
        - U: Left singular vectors (n x K)
        - Lambda: Singular values (K,)
        - Y_pred_train: Predicted Y for training data
        - rmse_train: RMSE for training data (regression)
        - Z_test: Dimension reduced test data (m x K), if X_test provided
        - Y_pred_test: Predicted Y for test data, if X_test and Y_test provided
        - rmse_test: RMSE for test data, if X_test and Y_test provided
    """
    # Input validation
    n, p = X.shape
    Y = Y.reshape(-1) if Y.ndim == 1 else Y[:, 0]

    # Standardize data if requested
    scaler_X, scaler_Y = None, None
    if standardize:
        scaler_X = StandardScaler()
        scaler_Y = StandardScaler()
        X = scaler_X.fit_transform(X)
        Y = scaler_Y.fit_transform(Y.reshape(-1, 1)).reshape(-1)

    # Step 1: Compute kernel matrix
    L = compute_kernel(Y, kernel_type, sigma)
    if L.shape != (n, n):
        raise ValueError("L must be n x n")

    # Step 2: Decompose L such that L = Delta^T Delta
    U_L, S_L, Vt_L = np.linalg.svd(L, hermitian=True)
    S_L = np.clip(S_L, 0, None)
    Delta = U_L @ np.diag(np.sqrt(S_L))

    # Step 3: Compute H = I - (1/n)e^T e
    e = np.ones((n, 1))
    H = np.eye(n) - (1/n) * (e @ e.T)

    # Step 4: Compute Psi = Delta^T H X
    Psi = Delta.T @ H @ X

    # Step 5: Compute sparse basis using PMD
    V = np.zeros((p, K))
    U = np.zeros((n, K))
    Lambda = np.zeros(K)
    Psi_k = Psi.copy()

    for k in range(K):
        # Initialize v_k with L2-norm = 1
        v_k = np.random.randn(p)
        v_k = v_k / np.linalg.norm(v_k)

        for iter_idx in range(max_iter):
            v_old = v_k.copy()

            # Update u_k
            u_k = Psi_k @ v_k
            if np.linalg.norm(u_k) < 1e-10:
                print(f"Warning: u_k norm too small at component {k+1}, iteration {iter_idx+1}")
                u_k = np.random.randn(n)

            # Orthogonalize u_k against previous u_j
            if k > 0:
                u_k = u_k - U[:, :k] @ (U[:, :k].T @ u_k)
                if np.linalg.norm(u_k) < 1e-10:
                    print(f"Warning: u_k norm too small after orthogonalization at component {k+1}")
                    u_k = np.random.randn(n)
                u_k = u_k / np.linalg.norm(u_k)

            # Update v_k
            a = Psi_k.T @ u_k
            tau = find_tau(a, c) 
            v_k = soft_threshold(a, tau)
            if np.linalg.norm(v_k) < 1e-10:
                print(f"Warning: v_k norm too small at component {k+1}, iteration {iter_idx+1}")
                v_k = np.random.randn(p)
            v_k = v_k / np.linalg.norm(v_k)

            # Check convergence
            norm_diff = np.linalg.norm(v_k - v_old)
            if norm_diff < tol * np.linalg.norm(v_old):
                if verbose:
                    print(f"Component {k+1} converged in {iter_idx+1} iterations")
                break
        else:
            print(f"Warning: PMD did not converge for component {k+1} after {max_iter} iterations")

        Lambda[k] = u_k.T @ Psi_k @ v_k
        U[:, k] = u_k
        V[:, k] = v_k

        # Deflate Psi
        Psi_k = Psi_k - Lambda[k] * np.outer(u_k, v_k)

    # Step 6: Encode training data
    Z = X @ V

    # Step 7: Regression for evaluation (assuming regression task)
    result = {
        'Z': Z,
        'V': V,
        #'U': U,
        #'Lambda': Lambda
    }

    if X_test is not None:
        #X_test = scaler_X.fit_transform(X_test)
        Z_test = X_test @ V
        result['Z_test'] = Z_test

    return result


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from scipy.linalg import eigh

# Read data
y = pd.read_csv("actual.csv")
data_2 = pd.read_csv("data_set_ALL_AML_independent.csv")
data_3 = pd.read_csv("data_set_ALL_AML_train.csv")
print(y.shape)
print(data_2.shape)
print(data_3.shape)

Y = y.replace({'ALL':0,'AML':1})
labels = ['ALL', 'AML']

train_to_keep = [col for col in data_3.columns if "call" not in col]
test_to_keep = [col for col in data_2.columns if "call" not in col]

X_train_tr = data_3[train_to_keep]
X_test_tr = data_2[test_to_keep]

train_columns_titles = ['Gene Description', 'Gene Accession Number', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38']

X_train_tr = X_train_tr.reindex(columns=train_columns_titles)

test_columns_titles = ['Gene Description', 'Gene Accession Number','39', '40', '41', '42', '43', '44', '45', '46',
       '47', '48', '49', '50', '51', '52', '53',  '54', '55', '56', '57', '58', '59',
       '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72']

X_test_tr = X_test_tr.reindex(columns=test_columns_titles)

X_train = X_train_tr.T
X_test = X_test_tr.T

X_train.columns = X_train.iloc[1]
X_train = X_train.drop(["Gene Description", "Gene Accession Number"]).apply(pd.to_numeric)

# Clean up the column names for Testing data
X_test.columns = X_test.iloc[1]
X_test = X_test.drop(["Gene Description", "Gene Accession Number"]).apply(pd.to_numeric)

X_train = X_train.reset_index(drop=True)
Y_train = Y[Y.patient <= 38].reset_index(drop=True)

# Subset the rest for testing
X_test = X_test.reset_index(drop=True)
Y_test = Y[Y.patient > 38].reset_index(drop=True)

X_train_fl = X_train.astype(float, 64)
X_test_fl = X_test.astype(float, 64)


#X_train_fl = X_train_fl.iloc[:, :500]
#X_test_fl = X_test_fl.iloc[:, :500]

# Apply the same scaling to both datasets
scaler = StandardScaler()
X_train_scl = scaler.fit_transform(X_train_fl)
X_test_scl = scaler.transform(X_test_fl)

Y_train = Y_train['cancer']
Y_test = Y_test['cancer']

(72, 2)
(7129, 70)
(7129, 78)


/var/folders/b6/0025j9bx74b7s62z1cxmw0ch0000gp/T/ipykernel_9627/1327986414.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y = y.replace({'ALL':0,'AML':1})


In [8]:
def run_all_methods_analysis_real_data(X_train, Y_train, X_test, Y_test, n_components_list=[2], threshold=0.25):
    p = X_train.shape[1]
    Y_train = np.array(Y_train).flatten()
    Y_test = np.array(Y_test).flatten()

    results = {
        'n_components': n_components_list,
        'precision_PCR': [],
        'error_PCR': [],
        'accuracy_PCR': [],
        'non_zero_vars_PCR': [],
        'auc_PCR': [],
        'precision_LDA': [],
        'error_LDA': [],
        'accuracy_LDA': [],
        'non_zero_vars_LDA': [],
        'auc_LDA': [],
        'precision_Barshan': [],
        'error_Barshan': [],
        'accuracy_Barshan': [],
        'non_zero_vars_Barshan': [],
        'auc_Barshan': [],
        'precision_Bair': [],
        'error_Bair': [],
        'accuracy_Bair': [],
        'non_zero_vars_Bair': [],
        'auc_Bair': [],
        'precision_Supervised': [],
        'error_Supervised': [],
        'accuracy_Supervised': [],
        'non_zero_vars_Supervised': [],
        'auc_Supervised': [],
        'precision_Sparse': [],
        'error_Sparse': [],
        'accuracy_Sparse': [],
        'non_zero_vars_Sparse': [],
        'auc_Sparse': [],
        'precision_SDSPCA': [],
        'error_SDSPCA': [],
        'accuracy_SDSPCA': [],
        'non_zero_vars_SDSPCA': [],
        'auc_SDSPCA': [],
        #'precision_SPCA': [],
        #'error_SPCA': [],
        #'accuracy_SPCA': [],
        #'non_zero_vars_SPCA': [],
        #'auc_SPCA': [],
        'precision_SSPCA': [],
        'error_SSPCA': [],
        'accuracy_SSPCA': [],
        'non_zero_vars_SSPCA': [],
        'auc_SSPCA': []
    }

    
    #para_spca_grid = [[0.1, 0.1]]    #,[0.1,0.1],[1,1]]  
    c_sspca_grid = [onp.sqrt(p)/32,onp.sqrt(p)/16,onp.sqrt(p)/8, onp.sqrt(p)/4, onp.sqrt(p)/2,]# onp.sqrt(p)]         

    
    X_train_main, X_val, Y_train_main, Y_val = train_test_split(
        X_train, Y_train, test_size=0.2, random_state=2002)

    scaler_X = StandardScaler()
    X_train_main = scaler_X.fit_transform(X_train_main)
    X_val = scaler_X.transform(X_val)
    X_test_scaled = scaler_X.transform(X_test)
    
    def count_nonzero_vars(W, threshold=1e-7):
        if W.ndim == 1:
            W = W.reshape(-1, 1)
        return onp.mean(onp.sum(onp.abs(W) > threshold, axis=0))

    for n_components in n_components_list:
        # PCA
        pca = PCA(n_components=n_components)
        X_train_pca = pca.fit_transform(X_train_main)
        X_test_pca = pca.transform(X_test_scaled)
        clf_pcr = LogisticRegression(solver='lbfgs', max_iter=1000)
        clf_pcr.fit(X_train_pca, Y_train_main)
        Y_pred = clf_pcr.predict(X_test_pca)
        Y_pred_proba = clf_pcr.predict_proba(X_test_pca)[:, 1]
        results['precision_PCR'].append(precision_score(Y_test, Y_pred, zero_division = 0))
        results['error_PCR'].append(1 - accuracy_score(Y_test, Y_pred))
        results['accuracy_PCR'].append(accuracy_score(Y_test, Y_pred))
        results['non_zero_vars_PCR'].append(p)
        results['auc_PCR'].append(roc_auc_score(Y_test, Y_pred_proba))
        
        # LDA
        n_comp_lda = min(n_components, len(np.unique(Y_train_main))-1, X_train_main.shape[1])
        lda = LinearDiscriminantAnalysis(n_components=n_comp_lda)
        X_train_lda = lda.fit_transform(X_train_main, Y_train_main)
        X_test_lda = lda.transform(X_test_scaled)
        Y_pred_lda = lda.predict(X_test_scaled)
        Y_pred_proba_lda = lda.predict_proba(X_test_scaled)[:, 1]
        results['precision_LDA'].append(precision_score(Y_test, Y_pred_lda, zero_division = 0))
        results['error_LDA'].append(1 - accuracy_score(Y_test, Y_pred_lda))
        results['accuracy_LDA'].append(accuracy_score(Y_test, Y_pred_lda))
        results['non_zero_vars_LDA'].append(p)
        results['auc_LDA'].append(roc_auc_score(Y_test,Y_pred_proba_lda))
        
        # HSIC
        K = delta_kernel(Y_train_main)
        spca_result = barshan_pca(X_train_main, Y_train_main, K, n_components)
        W_barshan = spca_result['W']
        Z_test_bar = X_test_scaled @ W_barshan
        clf_barshan = LogisticRegression(solver='lbfgs', max_iter=1000)
        clf_barshan.fit(X_train_main @ W_barshan, Y_train_main)
        Y_pred_bar = clf_barshan.predict(Z_test_bar)
        Y_pred_proba_bar = clf_barshan.predict_proba(Z_test_bar)[:, 1]
        results['precision_Barshan'].append(precision_score(Y_test, Y_pred_bar, zero_division = 0))
        results['error_Barshan'].append(1 - accuracy_score(Y_test, Y_pred_bar))
        results['accuracy_Barshan'].append(accuracy_score(Y_test, Y_pred_bar))
        results['non_zero_vars_Barshan'].append(count_nonzero_vars(W_barshan))
        results['auc_Barshan'].append(roc_auc_score(Y_test,Y_pred_proba_bar))

        # Bair's
        feature_scores = onp.abs(onp.corrcoef(X_train_main.T, Y_train_main.T)[:-1, -1])
        selected_features = feature_scores >= threshold
        X_train_selected = X_train_main[:, selected_features]
        X_test_selected = X_test_scaled[:, selected_features]
        pca_bair = PCA(n_components=min(n_components, X_train_selected.shape[1]))
        X_train_bair = pca_bair.fit_transform(X_train_selected)
        X_test_bair = pca_bair.transform(X_test_selected)
        clf_bair = LogisticRegression(solver='lbfgs', max_iter=1000)
        clf_bair.fit(X_train_bair, Y_train_main)
        Y_pred_bair = clf_bair.predict(X_test_bair)
        Y_pred_proba_bair = clf_bair.predict_proba(X_test_bair)[:, 1]
        results['precision_Bair'].append(precision_score(Y_test, Y_pred_bair, zero_division = 0))
        results['error_Bair'].append(1 - accuracy_score(Y_test, Y_pred_bair))
        results['accuracy_Bair'].append(accuracy_score(Y_test, Y_pred_bair))
        results['non_zero_vars_Bair'].append(onp.sum(selected_features))
        results['auc_Bair'].append(roc_auc_score(Y_test,Y_pred_proba_bair))

        # CSPCA
        lambda_grid = [0.01]
        best_lambda = lambda_grid[0]
        best_error = float('inf')
        for lambda_ in lambda_grid:
            spca_sup = supervised_pca(X_train_main, Y_train_main, n_components, p=p, m=145, lambda_=lambda_)
            W_sup = spca_sup['W']
            Z_val_sup = X_val @ W_sup
            clf_sup = LogisticRegression(solver='lbfgs', max_iter=1000)
            clf_sup.fit(X_train_main @ W_sup, Y_train_main)
            Y_pred_sup = clf_sup.predict(Z_val_sup)
            current_error = 1 - accuracy_score(Y_val, Y_pred_sup)
            if current_error < best_error:
                best_error = current_error
                best_lambda = lambda_
        
        spca_sup = supervised_pca(X_train_main, Y_train_main, n_components, p=p, m=145, lambda_=best_lambda)
        W_sup = spca_sup['W']
        Z_test_sup = X_test_scaled @ W_sup
        clf_sup = LogisticRegression(solver='lbfgs', max_iter=1000)
        clf_sup.fit(X_train_main @ W_sup, Y_train_main)
        Y_pred_sup = clf_sup.predict(Z_test_sup)
        Y_pred_proba_sup = clf_sup.predict_proba(Z_test_sup)[:, 1]
        results['precision_Supervised'].append(precision_score(Y_test, Y_pred_sup, zero_division = 0))
        results['error_Supervised'].append(1 - accuracy_score(Y_test, Y_pred_sup))
        results['accuracy_Supervised'].append(accuracy_score(Y_test, Y_pred_sup))
        results['non_zero_vars_Supervised'].append(count_nonzero_vars(W_sup))
        results['auc_Supervised'].append(roc_auc_score(Y_test,Y_pred_proba_sup))
        
        # SCSPCA 
        eta_sparse = 10000  
        kappa = 0.1
        epsilon = 1e-6
        n = Y_train_main.shape[0]
        K = delta_kernel(Y_train_main) + epsilon * np.eye(n)
        C = X_train_main.T @ K @ X_train_main + kappa * X_train_main.T @ X_train_main
        W_manpg, F_manpg, sparsity, time_manpg, iter_, flag_succ, num_linesearch, mean_ssn = manpg_orth_sparse(
            C, n_components, p, eta_sparse * onp.ones(p))
        
        X_test_proj = X_test_scaled @ W_manpg
        clf_sparse = LogisticRegression(solver='lbfgs', max_iter=1000)
        clf_sparse.fit(X_train_main @ W_manpg, Y_train_main)
        Y_pred_sparse = clf_sparse.predict(X_test_proj)
        Y_pred_proba_sparse = clf_sparse.predict_proba(X_test_proj)[:, 1]
        results['precision_Sparse'].append(precision_score(Y_test, Y_pred_sparse, zero_division = 0))
        results['error_Sparse'].append(1 - accuracy_score(Y_test, Y_pred_sparse))
        results['accuracy_Sparse'].append(accuracy_score(Y_test, Y_pred_sparse))
        results['non_zero_vars_Sparse'].append(count_nonzero_vars(W_manpg))
        results['auc_Sparse'].append(roc_auc_score(Y_test,Y_pred_proba_sparse))

        # SDSPCA
        alpha_grid = [10000,1000000,10000000]
        beta_grid = [0.0000000000000001,0.000000000000001,0.00000000000001,0.0000000000001,0.000000000001,0.00000000001]

        
        best_alpha = alpha_grid[0]
        best_beta = beta_grid[0]
        best_error = float('inf')
        for alpha in alpha_grid:
            for beta in beta_grid:
                sdspca = SDSPCA(n_components=n_components, alpha=alpha, beta=beta)
                Q_train = sdspca.fit_transform(X_train_main, Y_train_main)
                Q_val = sdspca.transform(X_val)
                clf = LogisticRegression(solver='lbfgs', max_iter=1000)
                clf.fit(Q_train, Y_train_main)
                Y_pred = clf.predict(Q_val)
                current_error = 1 - accuracy_score(Y_val, Y_pred)
                if current_error < best_error:
                    best_error = current_error
                    best_alpha = alpha
                    best_beta = beta       

        print(f"Best SDSPCA params: alpha={best_alpha}, beta={best_beta}")
        sdspca = SDSPCA(n_components=n_components, alpha=best_alpha, beta=best_beta)
        Q_train = sdspca.fit_transform(X_train_main, Y_train_main)
        Q_test = sdspca.transform(X_test_scaled)
        clf_sdspca = LogisticRegression(solver='lbfgs', max_iter=1000)
        clf_sdspca.fit(Q_train, Y_train_main)
        Y_pred_sdspca = clf_sdspca.predict(Q_test)
        Y_pred_proba_sdspca = clf_sdspca.predict_proba(Q_test)[:, 1]
        print(sdspca.Q_)
        results['precision_SDSPCA'].append(precision_score(Y_test, Y_pred_sdspca, zero_division = 0))
        results['error_SDSPCA'].append(1 - accuracy_score(Y_test, Y_pred_sdspca))
        results['accuracy_SDSPCA'].append(accuracy_score(Y_test, Y_pred_sdspca))
        results['non_zero_vars_SDSPCA'].append(count_nonzero_vars(sdspca.Q_))
        results['auc_SDSPCA'].append(roc_auc_score(Y_test,Y_pred_proba_sdspca))


        # SPCA 
        #best_para_spca = para_spca_grid[0]
        #best_error = float('inf')
        #for para_spca in para_spca_grid:
        #    spca_result = spca(X_train_main, K=n_components, 
        #         para=para_spca, 
        #         type_="predictor", sparse="penalty", lambda_=1e-4)
        #    W_spca = spca_result['loadings']
        #    Z_val_spca = X_val @ W_spca
        #    clf_spca = LogisticRegression(solver='lbfgs', max_iter=1000)
        #    clf_spca.fit(X_train_main @ W_spca, Y_train_main)
        #    Y_pred_spca = clf_spca.predict(Z_val_spca)
        #    current_error = 1 - accuracy_score(Y_val, Y_pred_spca)
        #    if current_error < best_error:
        #        best_error = current_error
        #        best_para_spca = para_spca

        #print(f"Selected para_spca: {best_para_spca}")
        #spca_result = spca(X_train_main, K=n_components, 
        #                 para=best_para_spca, 
        #                 type_="predictor", sparse="penalty", lambda_=1e-4)
        #W_spca = spca_result['loadings']
        #Z_test_spca = X_test_scaled @ W_spca
        #clf_spca = LogisticRegression(solver='lbfgs', max_iter=1000)
        #clf_spca.fit(X_train_main @ W_spca, Y_train_main)
        #Y_pred_spca = clf_spca.predict(Z_test_spca)
        #Y_pred_proba_spca = clf_spca.predict_proba(Z_test_spca)[:, 1]
        #results['precision_SPCA'].append(precision_score(Y_test, Y_pred_spca, zero_division = 0))
        #results['error_SPCA'].append(1 - accuracy_score(Y_test, Y_pred_spca))
        #results['accuracy_SPCA'].append(accuracy_score(Y_test, Y_pred_spca))
        #results['non_zero_vars_SPCA'].append(count_nonzero_vars(W_spca))
        #results['auc_SPCA'].append(roc_auc_score(Y_test,Y_pred_proba_spca))

        # SSPCA
        best_c_sspca = c_sspca_grid[0]  
        best_error = float('inf')
        for c_sspca in c_sspca_grid:
            sspca_result = sspca(X_train_main, Y_train_main, K=n_components, c=c_sspca, 
                   X_test=X_val, Y_test=Y_val, kernel_type='delta')
            Z_val_sspca = sspca_result['Z_test']
            clf_sspca = LogisticRegression(solver='lbfgs', max_iter=1000)
            clf_sspca.fit(sspca_result['Z'], Y_train_main)
            Y_pred_sspca = clf_sspca.predict(Z_val_sspca)
            current_error = 1 - accuracy_score(Y_val, Y_pred_sspca)
            if current_error < best_error:
                best_error = current_error
                best_c_sspca = c_sspca   
        
        print(f"Selected c_sspca: {best_c_sspca}")
        sspca_result = sspca(X_train_main, Y_train_main, K=n_components, c=best_c_sspca, 
                           X_test=X_test_scaled, Y_test=Y_test, kernel_type='delta')
        Z_test_sspca = sspca_result['Z_test']
        W_sspca = sspca_result['V']
        clf_sspca = LogisticRegression(solver='lbfgs', max_iter=1000)
        clf_sspca.fit(sspca_result['Z'], Y_train_main)
        Y_pred_sspca = clf_sspca.predict(Z_test_sspca)
        Y_pred_proba_sspca = clf_sspca.predict_proba(Z_test_sspca)[:, 1]
        results['precision_SSPCA'].append(precision_score(Y_test, Y_pred_sspca, zero_division = 0))
        results['error_SSPCA'].append(1 - accuracy_score(Y_test, Y_pred_sspca))
        results['accuracy_SSPCA'].append(accuracy_score(Y_test, Y_pred_sspca))
        results['non_zero_vars_SSPCA'].append(count_nonzero_vars(W_sspca))
        results['auc_SSPCA'].append(roc_auc_score(Y_test,Y_pred_proba_sspca))
    
    final_results = []
    
    for i, n_components in enumerate(n_components_list):
        row = {'n_components': n_components}
        
        for method in ['PCR', 'LDA', 'Barshan', 'Bair' ,'Supervised' , 'Sparse' , 'SDSPCA', 'SSPCA']:
            row[f'precision_{method}'] = results[f'precision_{method}'][i]
            row[f'error_{method}'] = results[f'error_{method}'][i]
            row[f'accuracy_{method}'] = results[f'accuracy_{method}'][i]
            row[f'non_zero_vars_{method}'] = results[f'non_zero_vars_{method}'][i]
            row[f'auc_{method}'] = results[f'auc_{method}'][i]
        
        final_results.append(row)
    
    return pd.DataFrame(final_results)

def analyze_real_dataset(X_train, Y_train, X_test, Y_test, n_components_list=[2]):
    results_df = run_all_methods_analysis_real_data(X_train, Y_train, X_test, Y_test, 
                                                  n_components_list=n_components_list)
    
    print("\nReal Dataset Analysis Results:")
    for method in ['PCR', 'LDA', 'Barshan', 'Bair', 'Supervised', 'Sparse', 'SDSPCA', 'SSPCA']:
        print(f"\nMethod: {method}")
        for n_comp in n_components_list:
            row = results_df[results_df['n_components'] == n_comp].iloc[0]
            print(f"  n_components = {n_comp}:")
            print(f"    Precision: {row[f'precision_{method}']:.4f}")
            print(f"    Classification Error: {row[f'error_{method}']:.4f}")
            print(f"    Accuracy: {row[f'accuracy_{method}']:.4f}")
            print(f"    Non-zero variables: {row[f'non_zero_vars_{method}']:.1f}")
            print(f"    AUC: {row[f'auc_{method}']:.1f}")
    
    return results_df

In [9]:
results = analyze_real_dataset(X_train_scl, Y_train, X_test_scl, Y_test, n_components_list=[2])

ManPG: Iter 999, Fval 4.18760e+04, CPU 565.15, Sparsity 1.00, Inner Inexact 1994, Avg Inner 101.00, Total Linesearch 13930
Best SDSPCA params: alpha=10000, beta=1e-16
[[-0.19987946 -0.04594855]
 [ 0.01180976 -0.38354536]
 [-0.20180091 -0.02109379]
 [ 0.01757462 -0.42248296]
 [-0.20338734 -0.00951083]
 [-0.2099622   0.04356725]
 [-0.20195461 -0.02583698]
 [-0.20210213 -0.02468345]
 [-0.19992137 -0.04741524]
 [ 0.01302857 -0.39097052]
 [-0.20258055 -0.02569972]
 [ 0.01091105 -0.37819898]
 [-0.19977052 -0.0423807 ]
 [-0.21173572  0.05279027]
 [-0.20861258  0.03084514]
 [-0.20326421 -0.01443573]
 [-0.19733531 -0.06611467]
 [-0.20349926 -0.01254665]
 [ 0.02261012 -0.46611671]
 [-0.20349487 -0.00604134]
 [ 0.01051344 -0.37533103]
 [-0.20558916  0.00577241]
 [-0.20477228  0.00192451]
 [-0.20848521  0.03091728]
 [-0.20569598  0.00713842]
 [-0.20496435  0.00084429]
 [-0.20378624 -0.00596562]
 [-0.20328915 -0.01501836]
 [-0.20492829  0.0049116 ]
 [-0.20420782 -0.00160603]]
Selected c_sspca: 2.63